In [48]:
import boto3
from botocore.exceptions import ClientError
import json
import sqlalchemy as sal
import pyodbc
import pandas as pd
import os

from dotenv import load_dotenv

load_dotenv()

True

In [49]:
def get_secret():

    secret_name = os.environ.get("AWS_SECRET_NAME")
    region_name = os.environ.get("AWS_REGION_NAME")

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    # Decrypts secret using the associated KMS key.
    secret = get_secret_value_response['SecretString']

    secret_json = json.loads(secret)
    return (secret_json['username'], secret_json['password'])

In [50]:
def get_sql_alquemy_engine():
    db_server_address = os.environ.get("AWS_DB_SERVER_ADDRESS")
    db_server_port = 1433
    db_name = 'tfm-indicators'
    username, password = get_secret()

    engine = sal.create_engine(f"mssql+pyodbc://{username}:{password}@{db_server_address}:{db_server_port}/{db_name}?driver=ODBC+Driver+17+for+SQL+Server",
                            fast_executemany = True)
    return engine

In [51]:
def insert_into_economic_calendar_table(df):
    i = 0
    engine = get_sql_alquemy_engine()
    with engine.connect() as connection:
        sql = "INSERT INTO [InvestingEconomicCalendar] (DateTime, Country, Currency, Event, Actual, Forecast, Previous) VALUES "

        for ind,row in df.iterrows():
            event = row['Event'].replace("'","")
            sql += f"('{row['DateTime']}','{row['Country']}','{row['Currency']}','{event}','{row['Actual']}','{row['Forecast']}','{row['Previous']}')"
            i += 1
            print('.', end='')
            
            if i == 100:
                connection.execute(sal.text(sql))
                sql = "INSERT INTO [InvestingEconomicCalendar] (DateTime, Country, Currency, Event, Actual, Forecast, Previous) VALUES "
                print('sent')
                i = 0
            else:
                sql += ", "

        if i > 0:
            sql = sql[:-2] # romove final coma
            connection.execute(sal.text(sql))
            print('sent')

        connection.commit()


In [52]:
df = pd.read_csv('economic_calendar.csv', parse_dates=True, index_col=0)
insert_into_economic_calendar_table(df)

C:\Users\lukas\AppData\Local\Temp\ipykernel_16200\2476591594.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('economic_calendar.csv', parse_dates=True, index_col=0)


....................................................................................................sent
....................................................................................................sent
....................................................................................................sent
....................................................................................................sent
....................................................................................................sent
....................................................................................................sent
....................................................................................................sent
....................................................................................................sent
....................................................................................................sent
.......................................................

In [45]:
engine = get_sql_alquemy_engine()
with engine.connect() as connection:
    connection.execute(sal.text("DELETE FROM InvestingEconomicCalendar"))
    connection.commit()

In [53]:
engine = get_sql_alquemy_engine()
with engine.connect() as connection:
    result = connection.execute(sal.text("select count(*) from InvestingEconomicCalendar"))
    for row in result:
        print(row)

(1068,)
